In [49]:
import pandas as pd
import numpy as np

In [50]:
# train = pd.read_csv("output/sorted_train0310.csv")
# test = pd.read_csv("output/sorted_test0310.csv")
# val = pd.read_csv("output/sorted_val0310.csv")

In [51]:
train = pd.read_csv("output/random_train0310.csv")
test = pd.read_csv("output/random_test0310.csv")
val = pd.read_csv("output/random_val0310.csv")

In [52]:
train = train[train['LoanStatus'] != 'MISSING']
test = test[test['LoanStatus'] != 'MISSING']
val = val[val['LoanStatus'] != 'MISSING']
print train['LoanStatus'].unique()
print test['LoanStatus'].unique()
print val['LoanStatus'].unique()

['PIF' 'CHGOFF']
['PIF' 'CHGOFF']
['CHGOFF' 'PIF']


In [53]:
# Compute some statistics
chargeoff_count = (train['LoanStatus'] == "CHGOFF").sum()
print 'Chargeoffs:', chargeoff_count
print 'Percentage of loans that defaulted:', float(chargeoff_count)/len(train)

chargeoff_count = (test['LoanStatus'] == "CHGOFF").sum()
print 'Chargeoffs:', chargeoff_count
print 'Percentage of loans that defaulted:', float(chargeoff_count)/len(test)

Chargeoffs: 7124
Percentage of loans that defaulted: 0.16248517471
Chargeoffs: 891
Percentage of loans that defaulted: 0.162561576355


In [54]:
####### optional ##########
# if import from sorted data - need to downsample train dataset to be more balanced
# can call the following function multiple times for decreasing in log term
# dropping PIF samples with 50% probability

def drop_half(train):
    print 'before drop: ', train.shape[0]
    dropidx = []
    for idx, val in train['LoanStatus'].iteritems():
        if val != 'CHGOFF':
            rand = np.random.rand()
            if rand < 0.5:
                dropidx.append(idx)
    train = train.drop(dropidx)
    print 'after drop: ', train.shape[0]
    return train
    
train = drop_half(train)
# train = drop_half(train)
# test = drop_half(test)

chargeoff_count = (train['LoanStatus'] == "CHGOFF").sum()
print 'Chargeoffs:', chargeoff_count
print 'Percentage of loans that defaulted:', float(chargeoff_count)/len(train)

chargeoff_count = (test['LoanStatus'] == "CHGOFF").sum()
print 'Chargeoffs:', chargeoff_count
print 'Percentage of loans that defaulted:', float(chargeoff_count)/len(test)

before drop:  43844
after drop:  25468
Chargeoffs: 7124
Percentage of loans that defaulted: 0.279723574682
Chargeoffs: 891
Percentage of loans that defaulted: 0.162561576355


In [55]:
# Define a function to generalize states to regions in order to increase frequency in each column 
West = ['AZ', 'CO', 'ID', 'MT', 'NV', 'NM', 'UT', 'WY', 'AK', 'CA', 'HI', 'OR', 'WA']
Midwest = ['IL', 'IN', 'MI', 'OH', 'WI', 'IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD']
South = ['DE', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'WV', 'DC', 'AL', 'KY', 'MS', 'TN', 'TX', 'OK', 'LA', 'AR']
Northeast = ['CT', 'ME', 'MA', 'NH', 'RI', 'VT', 'NJ', 'NY', 'PA']
Territory = ['GU', 'PR', 'VI']

def add_region_columns(df):
    in_west = []
    in_midwest = []
    in_south = []
    in_northeast = []
    in_territory = []
    for index, row in df.iterrows():
        state = row["BorrState"]
        if state in West:
            in_west.append(1)
            in_midwest.append(0)
            in_south.append(0)
            in_northeast.append(0)
            in_territory.append(0)
        elif state in Midwest:
            in_west.append(0)
            in_midwest.append(1)
            in_south.append(0)
            in_northeast.append(0)
            in_territory.append(0)
        elif state in South:
            in_west.append(0)
            in_midwest.append(0)
            in_south.append(1)
            in_northeast.append(0)
            in_territory.append(0)
        elif state in Northeast:
            in_west.append(0)
            in_midwest.append(0)
            in_south.append(0)
            in_northeast.append(1)
            in_territory.append(0)
        elif state in Territory:
            in_west.append(0)
            in_midwest.append(0)
            in_south.append(0)
            in_northeast.append(0)
            in_territory.append(1)
        else:
            print "Error - this state isn't in the regions:", state
        
    df['BorrRegion_West'] = pd.Series(in_west).values
    df['BorrRegion_Midwest'] = pd.Series(in_midwest).values
    df['BorrRegion_South'] = pd.Series(in_south).values
    df['BorrRegion_Northeast'] = pd.Series(in_northeast).values
    df['BorrRegion_Territory'] = pd.Series(in_territory).values
    df = df.drop('BorrState', axis = 1)
    
    return df

In [56]:
train = add_region_columns(train)
test = add_region_columns(test)

In [57]:
# filtering for the columns you don't want to include for your model
c = train.columns.tolist()
c.remove('LoanStatus')
c.remove('ChargeOffDate')
c.remove('GrossChargeOffAmount')
c.remove('BorrZip')
c.remove('CDC_Zip')
c.remove('BorrCity')
c.remove('CDC_City')
c.remove('ThirdPartyLender_City')
# c.remove('BorrState')
# c.remove('CDC_State')
# c.remove('ThirdPartyLender_State')
c.remove('ProjectCounty')
c.remove('ProjectState')
c.remove('ApprovalDate')
c.remove('CDC_State')
c.remove('ThirdPartyLender_State')
print c

['ApprovalFiscalYear', 'DeliveryMethod', 'subpgmdesc', 'TermInMonths', 'BusinessType', 'SP500_Yearly_Return', 'CPI', 'Log_GrossApproval_Norm', 'Log_HPI_Norm', 'ThirdPartyDollars_Norm', 'TermMultipleYear', 'RepeatBorrower', 'BankStateneqBorrowerState', 'ProjectStateneqBorrowerState', '2DigitNaics', 'BorrRegion_West', 'BorrRegion_Midwest', 'BorrRegion_South', 'BorrRegion_Northeast', 'BorrRegion_Territory']


In [58]:
x_train = train[c]
x_train = x_train.select_dtypes(exclude=['int64', 'float64'])
x_test = test[c]
x_test = x_test.select_dtypes(exclude=['int64', 'float64'])

y_train_logit = train['LoanStatus']
y_test_logit = test['LoanStatus']

# hazard_target = train['ChargeOffDate']
# amount_target = train['GrossChargeOffAmount']

# print train.dtypes
# convert categorical variables to dummy variables
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)
# val = pd.get_dummies(val)

In [67]:
y_train_logit = train['LoanStatus']
y_train_logit = [y_train_logit == 'CHGOFF']
y_train_logit = np.reshape(y_train_logit, (len(y_train_logit[0]),))

y_test_logit = test['LoanStatus']
y_test_logit = [y_test_logit == 'CHGOFF']
y_test_logit = np.reshape(y_test_logit, (len(y_test_logit[0]),))

In [68]:
# to get consistent feature dimensions for both train and test dataset
def add_missing_dummy_columns(d1, d2):
    missing_cols = set(d1.columns) - set(d2.columns)
    for c in missing_cols:
        d2[c] = 0
    return d2

print 'before fix columns: '
print x_train.shape
print x_test.shape

def fix_columns(x_train, x_test):  

    x_test = add_missing_dummy_columns(x_train, x_test)

    extra_cols = set(x_test.columns) - set(x_train.columns)
    x_test = x_test[x_train.columns]
    return x_test

x_test = fix_columns(x_train, x_test)
print 'after fix columns: '
print x_train.shape
print x_test.shape

before fix columns: 
(25468, 41)
(5481, 41)
after fix columns: 
(25468, 41)
(5481, 41)


In [69]:
# fit to models 
# below is to fit to logit model

# y_train_logit = np.reshape(l, (len(l),))
from sklearn.linear_model import LogisticRegression
solver = 'saga'

lr = LogisticRegression(solver=solver, 
                        multi_class= 'multinomial', 
                        C=1,
                        penalty = 'l1',
                        fit_intercept=True,
                        max_iter=10,
                        random_state=42, 
                       )
lr.fit(x_train, y_train_logit)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10, multi_class='multinomial',
          n_jobs=1, penalty='l1', random_state=42, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [62]:
y_train_logit[:20]

array([False, False, False, False, False,  True, False, False, False,
       False, False,  True,  True,  True, False, False,  True, False,
       False, False], dtype=bool)

In [74]:
# from sklearn.metrics import roc_curve, auc

# y_score = lr.decision_function(x_train)
# y_train_logit = pd.get_dummies(y_train_logit)

# # Compute ROC curve and ROC area for each class
# fpr = dict()
# tpr = dict()
# roc_auc = dict()
# for i in range(3):
#     fpr[i], tpr[i], _ = roc_curve(y_train_logit.loc[:,i], y_score[:,i])
#     roc_auc[i] = auc(fpr[i], tpr[i])
    
# # Compute micro-average ROC curve and ROC area
# fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
# roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# plt.figure()
# lw = 2
# plt.plot(fpr[2], tpr[2], color='darkorange',
#          lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
# plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic example')
# plt.legend(loc="lower right")
# plt.show()


In [70]:
lr.decision_function(x_train)
print lr.coef_

[[ 0.04845539  0.18328816 -0.20666078 -0.11500865 -0.33445047  0.20163807
   0.05489147 -0.33445047 -0.00250482  0.10642228  0.05489147  0.0131069
  -0.03151477  0.85605229 -0.21505827  0.02691115  0.         -0.00780962
   0.10000373  0.01830769  0.03753191 -0.1000586  -0.2631797   0.22280478
  -0.05728869 -0.02630272 -0.04323988  0.01573401  0.31519248  0.44154718
  -0.23635752  0.001677   -0.13625268  0.18796818 -0.1009836   0.37772968
   0.61056473  0.09801381  0.          0.         -0.47121602]]


In [71]:
# fit on training data to see if there is underfitting
y_pred_train = lr.predict(x_train)
print 'train: ', np.sum(y_pred_train == y_train_logit) *1./ y_train_logit.shape[0]

# fit on test set to see if there is overfitting
y_pred_test = lr.predict(x_test)
print 'test: ', np.sum(y_pred_test == y_test_logit) *1./ y_test_logit.shape[0]

train:  0.739987435213
test:  0.822295201606


In [72]:
from sklearn import metrics

print metrics.classification_report(y_test_logit, y_pred_test)

             precision    recall  f1-score   support

      False       0.86      0.93      0.90      4590
       True       0.42      0.25      0.31       891

avg / total       0.79      0.82      0.80      5481



In [73]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', penalty='l2',
              alpha=1e-3, random_state=42,
              max_iter=10, tol=None).fit(x_train, y_train_logit)
predicted = clf.predict(x_test)
np.mean(predicted == y_test_logit)   

0.81919357781426749